In [1]:
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
import shelve
from sklearn import metrics
import time
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
start = time.time()

### 读取样本

In [2]:
shelve_file = shelve.open('../data/sample')
X_train = shelve_file['X_train']
y_train = shelve_file['y_train']
X_test = shelve_file['X_test']

tfidf = TfidfVectorizer(ngram_range=(1,4), min_df=3, max_df=0.9, use_idf=1, smooth_idf=1, sublinear_tf=1)
tfidf.fit(X_train)
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
skb = SelectKBest(chi2,k = 7000000)#特征选择
X_train = skb.fit_transform(X_train,y_train) 
X_test = skb.transform(X_test) 
print(X_train.shape)

(100562, 7000000)


In [ ]:
clf1 = SGDClassifier(loss="modified_huber", alpha=0.00002, n_jobs=-1,penalty = 'l2')
clf2 = DecisionTreeClassifier()
# clf3 = LinearSVC(C=1.5, loss='squared_hinge', intercept_scaling=1, tol=0.0002, multi_class='ovr')
clf3 = SVC(kernel='linear',C=1.5,tol=0.0002,probability=True)
eclf = VotingClassifier(estimators=[('sgd', clf1), ('dtc', clf2), ('lsvc', clf3)], voting='soft')

/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### 训练

In [ ]:
train_start = time.time()
eclf.fit(X_train,y_train)
print("Time consumption on train",time.time( ) - train_start)

#训练效果
prediction = eclf.predict(X_train)
print(metrics.classification_report(y_train,prediction))#分类报告
print(metrics.confusion_matrix(y_train,prediction))#混淆矩阵

### 交叉验证

In [ ]:
vali_start = time.time()
scores = cross_val_score(eclf, X_train, y_train , cv=3, scoring='f1_macro')#交叉验证
print(scores) #各组分数
print(scores.mean(),'+/-',scores.std()*2) #平均分
print("Time consumption on cv",time.time( ) - vali_start)

### 预测

In [ ]:
test_start = time.time()
#预测
prediction = eclf.predict(X_test)
print('Time comsumption on test',time.time()-test_start)
#保存结果
f_out = open('voting.csv', 'w')
f_out.write("id,class"+"\n")
for i in range(len(X_test)):
    f_out.write(str(i)+","+str(prediction[i])+'\n')
f_out.close()
print('total time comsumption',time.time()-start)